# Step 5: Produce
Creates a TLDR structure for each Request for each day. The TLDR structure consists of a TLDR object linked to a set of TldrEntries with a score.

The result of this step includes:
- TldrEntry nodes connected to a Summary node with an INCLUDES relationship, and Recommendation nodes with a BASED_ON relationship
- TLDR nodes connected to Request nodes with a RESPONSE_TO relationship, and a set of TldrEntry nodes with a PRIORITY relationship (scored for ordering)

In [ ]:
import logging
from datetime import date

## Parameters
OpenTLDR workflows use the notebook block tagged as "parameters" to inject variables (for example where to store TLDR outputs).

> **Do Not Change Variable Names in the Parameters Block** you are welcome to change the values of these parameter variables, but please do not change their names. They are used elsewhere in the notebook and in other workflow processes.

In [ ]:
#Parameters
# Logging level ranges are (from least to most verbose): ERROR, WARN, INFO, DEBUG
logging_level= logging.INFO

# List of the UniqueIds of Requests to add summaries
list_of_uids = None

# level of unnecessary output
verbose = True


In [ ]:
logging.getLogger("OpenTLDR").setLevel(logging_level)

from opentldr import KnowledgeGraph
kg=KnowledgeGraph()

from opentldr.Domain import Tldr, TldrEntry, Recommendation, Summary

In [ ]:
if list_of_uids is None:
    # default to getting all Requests
    list_of_uids = kg.get_all_node_uids_by_tag("Request")

# Produce a TDRL for each Request


In [ ]:
today = date.today()

# Produce a TLDR for each Request in the Knowledgebase
for request_uid in list_of_uids:
    request = kg.get_request_by_uid(request_uid)

    # New TLDR for each day
    tldr=kg.add_tldr(request,today)
    print("\nTLDR for {title} on {today}\nRequest: {request}".format(title=request.title,today=today,request=request.text))

    # If an TLDR already exists for this day, remove previouse Entries
    kg.delete_tldr_entries_by_tldr(tldr)

    # Generate an Entry for each Recommendation that relates_to this Request
    recommendation_list = kg.get_recommendations_by_request(request)
    for recommendation in recommendation_list:
        #Include the summary that focus_on this Recommendation
        summary_list=kg.get_summaries_by_recommendation(recommendation)

        if summary_list is not None and len(summary_list) > 0:
            # The schema allows for mulitple summaries, but we only need one
            summary=summary_list[0] # just using first one
            original_content=summary.summarizes.single()

            # Create a new Entry for this TLDR
            tldr_entry=kg.add_entry_to_tldr(tldr=tldr,
                score=recommendation.score,
                recommendation=recommendation,
                summary=summary,
                content=original_content)
            
            # Produce some info and print the entries out
            original_size=len(original_content.text)
            summary_size=len(summary.text)
            #reduction = round(((original_size-summary_size)/original_size)*100,1)

            print("\u2022 {title} [{url}]: {summary}".format(
                summary=summary.text,
                title=original_content.title, 
                url=original_content.url
            ))
        else:
            print("No summary found for TLDR entry.")

In [ ]:
kg.close()